In [127]:
import pandas as pd
import numpy as np
from bitfile import *
from constants import *

In [128]:
ENC_FILE_PATH = "/Users/arroyoruizj/Desktop/comps_f25_rgj/dnazip/data/output/HG004_GRCh38_Encoded.bin"

In [129]:
TWO_BIT_ENCODING = {
    "00": "A",
    "01": "C",
    "10": "G",
    "11": "T",
}

In [130]:
def decode(file_to_bin_file):

    with open(file_to_bin_file, 'rb') as f:
        # Read the entire file content as a bytes object
        binary_data = f.read()

        bit_string = BytesToBitString(binary_data)
        
        padding = 0

        while bit_string[padding] != '1':

            padding += 1

        bitmap_size, bit_string = readBitVINT(bit_string[padding:])
        print(bitmap_size)

        bitmap = bit_string[:bitmap_size]
        bit_string = bit_string[bitmap_size:]
        snp_size, bit_string = readBitVINT(bit_string)
        
        snp_pos = []
        snp_nucs = []

        for i in range(snp_size):

            pos, bit_string = readBitVINT(bit_string)
            snp_pos.append(pos)

        print("SNP Positions Decoded")

        for i in range(0, snp_size * 2, 2):

            two_bits = bit_string[i:i+2]
            nuc = TWO_BIT_ENCODING[two_bits]
            snp_nucs.append(nuc)

        print("SNP Nucleotides Decoded")

        bit_string = bit_string[snp_size * 2:]

        # while bit_string[padding] != '1':

        #     padding += 1

    
        del_size, bit_string = readBitVINT(bit_string)
        print(del_size)



        # bitmap = bit_string[:bitmap_size]

        # # this part will need to figure out how we encode the alignments bits
        # # we could always search for the next one, though not guranteed with vints (vint that only needs one bit)
        # # maybe this is why the chr flag is important !!!
        # bit_string = bit_string[bitmap_size + 5:]



       

    

In [131]:
decode(ENC_FILE_PATH)

742467
SNP Positions Decoded
SNP Nucleotides Decoded
26176
